#### Краулер

Собираем отзывы на Майнкрафт на metacrics

In [142]:
import requests
import time
import random
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

positive = []
negative = []

session = requests.session()

ua = UserAgent(verify_ssl=False)
headers = {"User-Agent": ua.random}

random.uniform(1, 3)

for num in range(19):
    for _ in range(5):
        response = session.get(f"https://www.metacritic.com/game/pc/minecraft/user-reviews?page={num}", headers = headers)
        time.sleep(random.uniform(1.1, 5.2))

    page = response.text
    soup = BeautifulSoup(page, "html.parser")
    page_text = soup.find_all("div", {"class": "review_section"})

    for i in page_text:
        try: # избегаем ошибки None has no attribute text, когда встречаем смешанные отзывы
            if i.find("div", {"class": "metascore_w user medium game positive indiv"}):
                try:
                    positive.append(i.find("span", {"class": "blurb blurb_expanded"}).text) # если есть полная версия отзыва, пытаемся скачать её
                except:
                    positive.append(i.find("div", {"class": "review_body"}).text)
            elif i.find("div", {"class": "metascore_w user medium game negative indiv"}):
                try:
                    negative.append(i.find("span", {"class": "blurb blurb_expanded"}).text) # если есть полная версия отзыва, пытаемся скачать её
                except:
                    negative.append(i.find("div", {"class": "review_body"}).text)
        except:
            continue

#### Уравниваем кол-во отзывов, откладывыем по 10 из каждого списка на тест

In [143]:
positive = positive[:len(negative)] # уравниваем кол-во текстов

In [144]:
test = {}
for i in range(-1, -11, -1):
    test[positive.pop(i)] = "positive"
    test[negative.pop(i)] = "negative"

#### Предобработка с помощью spaCy

In [145]:
!-m pip install spacy
!-m spacy download en
!python -m spacy download en_core_web_sm
import spacy

nlp = spacy.load("en_core_web_sm")
lemmatizer = nlp.get_pipe("lemmatizer")

"-m" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"-m" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


     ---------------------------------------- 12.8/12.8 MB 2.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [146]:
def preprocessing(list): # мы работаем со списками, удобнее написать функцию, на вход которой можно сразу подать список
    preprocessed_words = []
    for text in list:
        preprocessed_text = nlp(text.lower()) # здесь текст уже делится по предложениям и токенизируется
        for word in [token.lemma_ for token in preprocessed_text if not token.is_punct]:
            if word.isalpha:
                preprocessed_words.append(word)
    return preprocessed_words

Считаем кол-во слов в каждом списке, убираем те, которые встречаются менее трёх раз

In [147]:
from collections import Counter

In [148]:
def deleting_unpopular(preprocessed_list):
    word_list = []
    cnt_dict = Counter(preprocessing(preprocessed_list))
    for i in cnt_dict:
        if cnt_dict[i] > 2:
            word_list.append(i)
    return set(word_list)

In [149]:
positive_set = deleting_unpopular(positive)

In [150]:
negative_set = deleting_unpopular(negative)

Удаляем слова, которые есть и в позитивных, и в негативных отзывах

In [151]:
intersection = positive_set.intersection(negative_set)
positive_set.difference_update(intersection)
negative_set.difference_update(intersection)

Функция для определения тональности:

In [152]:
freq_lists = {}

freq_lists["positive"] = positive_clean
freq_lists["negative"] = negative_dict

In [159]:
def simple_sent_analysis(freq_lists, text):
    preprocessed_review_list = []
    preprocessed_review = nlp(text.lower()) # здесь текст уже делится по предложениям и токенизируется
    for word in [token.lemma_ for token in preprocessed_review if not token.is_punct]:
        if word.isalpha:
            preprocessed_review_list.append(word)
    counts = Counter()
    for tone, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in preprocessed_review_list:
            counts[tone] += int(freq_list[word] > 0)
    return counts.most_common()[0][0]

Запущенная функция + тональность отзыва:

In [160]:
for i in test:
    print(simple_sent_analysis(freq_lists, i), test[i])

negative positive
negative negative
negative positive
positive negative
positive positive
negative negative
positive positive
negative negative
negative positive
positive negative
positive positive
negative negative
positive positive
negative negative
positive positive
positive negative
positive positive
positive negative
positive positive
negative negative


Подсчёт accuracy

In [155]:
!pip install sklearn

You should consider upgrading via the 'C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [156]:
from sklearn.metrics import accuracy_score

In [157]:
def test_simple_sent_analysis(freq_lists, test_list):
    results = []
    gold = []
    for tone in freq_lists:
        for text in test_list:
            predicted_tone = simple_sent_analysis(freq_lists, text)[0][0]
            results.append(predicted_tone)
            gold.append(test_list[text])
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [158]:
test_simple_tone_detect(freq_lists, test)

RESULTS:
Accuracy: 0.6500
